In [1]:
from my_tradertrader import *

C:\Users\gli26\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def get_price_data(tic_list,method,interval = 1, freq = 'minutes',start_date = datetime.now()-timedelta(days =256),end_date=datetime.now()):

    def get_ondemand_data(sym, interval = 1,freq = 'minutes' ,start_date=(datetime.now()-timedelta(days=10))\
                    ,end_date=datetime.now()):

        try:

            api_key = '95b5894daf3abced33fe48e7f265315e'
            start_date=start_date.strftime("%Y%m%d%H%M%S")
            end_date=end_date.strftime("%Y%m%d%H%M%S")
            # This is the required format for datetimes to access the API

            api_url = 'http://marketdata.websol.barchart.com/getHistory.csv?' + \
                                    'key={}&symbol={}&type={}&startDate={}&endDate={}&interval={}'\
                                     .format(api_key, sym, freq, start_date,end_date,interval)

            csvfile = pd.read_csv(api_url, parse_dates=['timestamp'])
            csvfile.set_index('timestamp', inplace=True)

            return csvfile

        except Exception as e:
            print e

    error = []
    price = pd.DataFrame()
    if method == "robinhood":
        robinhood = get_robinhood()

        for i in tic_list:
            trial = 0
            while trial <3:
                try:
                    temp = robinhood.get_historical(i,interval="10minute",span = "week")
                    #temp.index = temp.begins_at
                    temp = temp.drop(["session","interpolated"],axis=1)
                    for j in temp.columns[1:]:
                        temp[j] = temp[j].astype(float)
                   # index= pd.MultiIndex.from_product([[i],temp.index])
                    temp["Ticker"] = np.repeat(i,len(temp))
                    price = price.append(temp)

                    print "Finished", i
                    #time.sleep(5)
                    trial=3

                except:
                    print "error occorded in getting yahool historicals for ", i
                    trial +=1
                    time.sleep(10)
                    if trial == 3:
                        error.append([i,'get_yahoo_historicals'])
        # get rid of the multiindex
       # price.to_csv("file/temp.csv")
       # price = pd.read_csv("file/temp.csv")

        price = price.reset_index()
        price["Close"] = price["close_price"]
        price = price.rename(columns={'begins_at':"TimeStamp","high_price":"High","low_price":"Low","open_price":"Open","volume":"Volume"})
        price["Return"]= (price.Close.diff(1)/price.Close)


    elif method == "ondemand":


        #freq = minutes, daily
        save_file_name = "Trade_suggestion_ondemand"
        for i in tic_list:
            trial = 0
            while trial <3:
                try:
                    temp = get_ondemand_data(i, interval = interval , freq = freq,start_date = start_date, end_date = end_date)

                    #index= pd.MultiIndex.from_product([[i],temp.index])
                    #temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)

                    price = price.append(temp)

                    print "Finished", i
                    #time.sleep(5)
                    trial=3

                except:
                    print "error occorded in getting yahool historicals for ", i
                    trial +=1
                    time.sleep(10)
                    if trial == 3:
                        error.append([i,'get_yahoo_historicals'])


        price = price.reset_index()
        price["Close"] = price["close"]
        price = price.rename(columns={'symbol':'Ticker','timestamp':"TimeStamp","high":"High","low":"Low","open":"Open","volume":"Volume"})
        price["Return"]=( price.Close.diff(1)/price.Close)

    elif method == "day":
        save_file_name = "Trade_suggestion_day"
        mongodb = mongo()
        for i in tic_list:
            trial = 0
            while trial <3:
                try:
                    temp = mongodb.query_database(i,start_date=start_date,end_date=end_date)
                    index= pd.MultiIndex.from_product([[i],temp.index])
                    temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)
                    price = price.append(temp)
                    print "Finished", i
                    trial = 3
                except Exception as e:
                    try:
                        print (e)
                        print ("Switching from Mongodb to yahoo")
                        temp = da.DataReader(i,"yahoo",start_date ,end_date)
                        index= pd.MultiIndex.from_product([[i],temp.index])
                        temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)
                        price = price.append(temp)

                        print "Finished", i
                        #time.sleep(5)
                        trial=3


                    except Exception as e:
                        print "error occorded in getting yahool historicals for ", i
                        trial =3
                        trial +=1
                        time.sleep(10)
                    if trial == 3:
                        error.append([i,'get_yahoo_historicals'])
        # get rid of the multiindex
    elif method == "realtimeday":
        save_file_name = "Trade_suggestion_day"
        mongodb = mongo()
        for i in tic_list:
            trial = 0
            while trial <3:
                try:
                    temp = pdr.get_data_yahoo(i,start_date ,end_date)
                    index= pd.MultiIndex.from_product([[i],temp.index])
                    temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)
                    price = price.append(temp)
                    print "Finished", i
                    trial = 3
                except Exception as e:
                    try:
                        print (e)
                        print "Finished", i
                        #time.sleep(5)
                        trial=3
                        print e

                    except Exception as e:
                        print "error occorded in getting yahool historicals for ", i
                        trial =3
                        trial +=1
                        time.sleep(10)
                    if trial == 3:
                        error.append([i,'get_yahoo_historicals'])
        price = price.reset_index()


        price["Close"] = price["Adj Close"]
        price = price.rename(columns={'level_0':'Ticker','level_1':"TimeStamp"})
        price["Return"]= (price.Close.diff(1)/price.Close)
    #price = price.dropna()
    return price

In [14]:
get_price_data(["AAPL"], method="realtimeday")

[*********************100%***********************]  1 of 1 downloaded
Finished AAPL


,Ticker,TimeStamp,Open,High,Low,Close,Adj Close,Volume,Return
0,AAPL,2018-01-08,174.350006,175.610001,173.929993,172.370514,172.370514,20567800.0,NaN
1,AAPL,2018-01-09,174.550003,175.059998,173.410004,172.350739,172.350739,21584000.0,-0.000115
2,AAPL,2018-01-10,173.160004,174.300003,173.000000,172.311188,172.311188,23959900.0,-0.000230
3,AAPL,2018-01-11,174.589996,175.490005,174.490005,173.289948,173.289948,18667700.0,0.005648
4,AAPL,2018-01-12,176.179993,177.360001,175.649994,175.079391,175.079391,25226000.0,0.010221
5,AAPL,2018-01-16,177.899994,179.389999,176.139999,174.189636,174.189636,29565900.0,-0.005108
6,AAPL,2018-01-17,176.149994,179.250000,175.070007,177.066589,177.066589,34386800.0,0.016248
7,AAPL,2018-01-18,179.369995,180.100006,178.250000,177.224762,177.224762,31193400.0,0.000892
8,AAPL,2018-01-19,178.610001,179.580002,177.410004,176.433853,176.433853,32425100.0,-0.004483
9,AAPL,2018-01-22,177.300003,177.779999,176.600006,174.990433,174.990433,27108600.0,-0.008249
